In [31]:
import sys
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn import set_config; set_config(display='diagram')

sys.path.append("/Users/antonis/code/Ant-mel/legendary_game_recs/")
from preprocessing.preprocess_1_cleaning import *
from sklearn.compose import ColumnTransformer

from sklearn.pipeline import make_pipeline
from sklearn.pipeline import make_union
from sklearn.compose import make_column_transformer



In [4]:
data = pd.read_csv('../raw_data/all_game_data_v1_corrected2', low_memory=False)

In [40]:
data['genres'][0]

"['Indie', 'Racing', 'Sport']"

In [55]:
def remove_no_reviews(df):
    df_good_mask = df['avg_review'] != 0
    df_with_reviews = df[df_good_mask]
    return df_with_reviews

def test(df_column):
    for i, x in df_column:
        x_new_slice = x[2:-2]
        x_new_replace = x_new_slice.replace("'", '').split(',')
        
        df_column[i] = x_new_replace
    
    return df_column
        

#Ant Reworking functions below for pipeline
def make_stringlist_list2(series):
    '''This removes square brackets, and splits the string by comma to the create a list'''
    for x in series:
        string = series[x]
        list_of_strings = string[2:-2].replace("'", '').split(',')
        return list_of_strings

# def remove_whitespace2(list_):
#     '''This removes whitesapces from items within lists'''
#     empty = []

#     for i in range(len(list_)):
#         item = list_[i].strip()
#         empty.append(item)
#     return empty

# def clean_stringlists2(df):
#     '''For lists that were imported as strings, this removes square brackers and cleans up trailing whitespaces'''
#     string_to_list_df = df.apply(make_stringlist_list2)
#     remove_whitespace_df = string_to_list_df.apply(remove_whitespace2)

#     return remove_whitespace_df

In [32]:
drop_dupes = FunctionTransformer(drop_duplicates)
remove_0_reviews = FunctionTransformer(remove_no_reviews)
clean_strings = FunctionTransformer(clean_stringlists2)


pipeline = Pipeline([('drop_duplicates', drop_dupes),
                     ('remove_no_reviews', remove_0_reviews)])

column_cleaner = ColumnTransformer([('making_categories_lists', clean_strings, ['genres', 'developers', 'platforms'])],remainder='passthrough')

pipeline2 = make_pipeline(pipeline, column_cleaner)